In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
from requests import Session
import requests

import pandas as pd
import time
import datetime
from tqdm import tqdm
import re
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))

# db management libraries
import pymysql
from controller import MysqlController

from diningcode_crawling import *

In [3]:
!pwd

/Users/yejin/Desktop/MealGen_crawling/DiningCode_crawling


In [13]:
# 연결
with open('../connection.txt', "r") as f:
                connect_info = list(map(lambda x: x.strip(), f.read().split(",")))
server = MysqlController(*connect_info)

# 주소지 받아오기
# ADR = pd.read_sql('SELECT DISTINCT CONCAT(sigungu, " ",dong) as adr FROM Address;', server.conn)
# server.curs.execute('SELECT DISTINCT CONCAT(sigungu, " ",dong) as adr FROM Address;')
# ADR = [f[0] for f in server.curs.fetchall() if re.sub('세종특별자치시 ([가-힇]+[면읍\)]|[(]알수없음[)])$', '', str(f[0])) != ""]

In [12]:
server.conn.close()

# 리뷰 중복제거

In [14]:
revs = pd.read_sql('SELECT * FROM diningcode_reviews ORDER BY created_at ASC;', server.conn)

In [15]:
len(revs)

53178

In [16]:
new_revs = revs.drop_duplicates(subset = ['rname', 'rid', 'reviewer', 'reviewer_info', 'star', 'date', 'review'], keep = 'first')
len(new_revs)

52969

In [10]:
for id in set(revs._id) - set(new_revs._id):
    server.curs.execute(f'DELETE FROM diningcode_reviews WHERE _id = {id};')
    server.curs.execute(f'UPDATE diningcode_reviews SET updated_at = now() WHERE _id = {id};')
    server.conn.commit()
# set(revs._id) - set(new_revs._id)

# 식당 리스트 test

In [13]:
server = DiningCode("../connection.txt")

In [14]:
server.driver.get("https://diningcode.com/list.php?query=세종특별자치시 연동면")

In [20]:
server.driver.find_element_by_css_selector('#contents > div.localeft-cont > div:nth-child(3)').click()